# **Baseline Code**

In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import load_model, Model
import tensorflow as tf
import keras

class LeNetCNN:
  # constructor
  def __init__(self):
    self.model = None

  # Define structure of the CNN
  def build(self, input_dim, use_act=True, use_pool=True):
    input = Input(shape = input_dim) # X dau vao cua minh

    if use_act:
      out = Conv2D(6, (3, 3), padding='same', activation='relu')(input)
    else:
      out = Conv2D(6, (3, 3), padding='same')(input)

    if use_pool:
      out = MaxPooling2D(pool_size=(2, 2), padding="same")(out)

    if use_act:
      out = Conv2D(16, (3, 3), padding='same', activation='relu')(out)
    else:
      out = Conv2D(16, (3, 3), padding='same')(out)
    
    if use_pool:
      out = MaxPooling2D(pool_size=(2, 2), padding="same")(out)

    flat = Flatten()(out)
    
    # These like ANN
    if use_act:
      out = Dense(120, activation='relu', use_bias=True)(flat)
    else:
      out = Dense(120, use_bias=True)(flat)
    
    if use_act: 
      out = Dense(84, activation='relu', use_bias=True)(out)
    else:
      out = Dense(84, use_bias=True)(out)
      
    output = Dense(10, activation='softmax', use_bias=True)(out) # y~ output
    self.model = Model(input, output)
  
  # Train the model
  def train(self, x_train, y_train, x_val, y_val): # x_train chinh la X, y_train chinh la ground-truth
    sgd = keras.optimizers.SGD(learning_rate=0.01)
    adam = keras.optimizers.Adam(learning_rate=0.01)
    self.model.compile(optimizer=sgd, loss = 'categorical_crossentropy', metrics=['accuracy'])
    self.model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 50, batch_size = 128)
  
  # Load model from file
  def load(self, model_file):
    self.model = load_model(model_file)
  
  # save the trained model
  def save(self, model_file):
    self.model.save(model_file)
  
  # Show the architecture of the model
  def summary(self):
    self.model.summary()

  # Test the model with a given input
  def predict(self, x_test):
    return self.model.predict(x_test)



In [2]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
import numpy as np

def onehot(y):
  oh = np.zeros((y.shape[0], 10))
  for i in range(y.shape[0]):
    oh[i, int(y[i])]=1
  return oh

In [4]:
import numpy as np
def norm_zero_one(x_train):
  min_val = np.min(x_train)
  max_val = np.max(x_train)
  x_train_norm = (x_train-min_val)/(max_val-min_val)
  return x_train_norm

def norm_normal_dist(x_train):
  mean_val = np.mean(x_train)
  std_val = np.std(x_train)
  x_train_norm = (x_train-mean_val)/std_val
  return x_train_norm

In [5]:
from sklearn import preprocessing
import numpy as np

#enc = preprocessing.OneHotEncoder()

#enc.fit(y_train)

#y_train_oh = enc.transform(y_train).toarray()
#y_train_oh.shape
y_train_oh = onehot(y_train)
y_train_oh.shape

y_test_oh = onehot(y_test)
y_test_oh.shape

x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

#x_train_norm = x_train_norm[:,:,:, np.newaxis] # Optional: Convert 60000x28x28 -> 60000x28x28x1
#x_test_norm = x_test_norm[:,:,:, np.newaxis] # Optional: Convert 60000x28x28 -> 60000x28x28x1

In [6]:
x_train_norm.shape

(60000, 28, 28)

# **Standard CNN**

In [7]:
cnn = LeNetCNN()
cnn.build((28, 28, 1))
cnn.summary()
cnn.train(x_train_norm, y_train_oh, x_test_norm, y_test_oh)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 6)         60        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        880       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 784)               0     

# **Non Activation Funciton**

In [8]:
cnn2 = LeNetCNN()
cnn2.build((28, 28, 1), use_act=False)
cnn2.summary()
cnn2.train(x_train_norm, y_train_oh, x_test_norm, y_test_oh)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 6)         60        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 6)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 16)        880       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 784)               0   

# **Non Pooling Layer**

In [9]:
cnn3 = LeNetCNN()
cnn3.build((28, 28, 1), use_pool=False)
cnn3.summary()
cnn3.train(x_train_norm, y_train_oh, x_test_norm, y_test_oh)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 6)         60        
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 16)        880       
                                                                 
 flatten_2 (Flatten)         (None, 12544)             0         
                                                                 
 dense_6 (Dense)             (None, 120)               1505400   
                                                                 
 dense_7 (Dense)             (None, 84)                10164     
                                                                 
 dense_8 (Dense)             (None, 10)                850 

# **Kết luận**

> Việc loại bỏ **Activation Function** giảm độ chính xác của mô hình.

> Việc loại bỏ **Pooling Layer** tuy làm mô hình train lâu hơn, nhưng giúp tăng độ chính xác.